In [1]:
# Imports 

import requests
import json
import pandas as pd

In [2]:
# Get the Response Code
# Storing Parameters
params = (
    ('contains', 'engineering and technology'),
)
response = requests.get('http://api.dataatwork.org/v1/skills/autocomplete', params=params)

# converting to the json object
response_json = response.json()

# Creating a dataframe to store the response 
df = pd.DataFrame(response_json)
df


,uuid,suggestion,normalized_skill_name
0,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology


In [3]:
# Storing the value of Skill UUID in a variable
Skill_UUID = []
for i in range (len(df)):
  Skill_UUID.append(df.iloc[i,0])
print(Skill_UUID)

['c09c8f8c163907ac25a37c3dd591ba2e']


In [4]:
# Storing the value of Skill UUID in a variable

skillUUID = df.iloc[0,0]
print(skillUUID)

c09c8f8c163907ac25a37c3dd591ba2e


In [5]:
# Now we have to get the associated jobs for the skill (engineering and technology). 
# So the input to the URL will be the uid of skill (skillUUID)


response2 = requests.get("http://api.dataatwork.org/v1/skills/"+skillUUID+"/related_jobs")

#verifying the correct URL
response2.url

'http://api.dataatwork.org/v1/skills/c09c8f8c163907ac25a37c3dd591ba2e/related_jobs'

In [6]:
response2_json = response2.json()

# Creating a dataframe to store the response 
df2 = pd.DataFrame(response2_json)
df2

,skill_uuid,skill_name,normalized_skill_name,jobs
0,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '91e03483b47fc212441abba3acf5a4aa...
1,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '19a7f05a1ba98db9e72086ebc8bd3ae0...
2,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': 'b8a5203799a5b6a9164b73170777fa11...
3,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '351a52f8a687142e32acd0b6c331e61b...
4,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '84f4c3f6e9e59cf9c4a840f6b5f2ebb3...
...,...,...,...,...
949,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '998a71de29e992c5dc61cd79c09b6cd7...
950,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '2311892cb8cf483f3b6a17489be5057d...
951,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '90c550c8805a100bd96937740668cbc8...
952,c09c8f8c163907ac25a37c3dd591ba2e,engineering and technology,engineering and technology,{'job_uuid': '7bbe67cb54b2e639a6a6e3fe505e54bb...


In [7]:
df2.skill_uuid.unique()
# verification - this dataframe contains all the associated jobs related to only one skill

array(['c09c8f8c163907ac25a37c3dd591ba2e'], dtype=object)

In [8]:
df2.jobs[1]

{'importance': 4.92,
 'job_title': 'Electronics Engineers, Except Computer',
 'job_uuid': '19a7f05a1ba98db9e72086ebc8bd3ae0',
 'level': 5.7,
 'normalized_job_title': 'electronics engineers except computer'}

In [9]:
# jobs column is in a nested json format.
# To get the list of all the jobs (ie- job title), we have to flatten/normalize the json object.

from pandas import json_normalize
df_jobs = json_normalize(data=df2['jobs'])
df_jobs

,job_uuid,job_title,normalized_job_title,importance,level
0,91e03483b47fc212441abba3acf5a4aa,Computer Hardware Engineers,computer hardware engineers,4.95,6.35
1,19a7f05a1ba98db9e72086ebc8bd3ae0,"Electronics Engineers, Except Computer",electronics engineers except computer,4.92,5.70
2,b8a5203799a5b6a9164b73170777fa11,Automotive Engineers,automotive engineers,4.91,6.39
3,351a52f8a687142e32acd0b6c331e61b,Aerospace Engineering and Operations Technicians,aerospace engineering and operations technicians,4.91,6.35
4,84f4c3f6e9e59cf9c4a840f6b5f2ebb3,Mechanical Engineers,mechanical engineers,4.89,6.08
...,...,...,...,...,...
949,998a71de29e992c5dc61cd79c09b6cd7,"Law Teachers, Postsecondary",law teachers postsecondary,1.02,0.02
950,2311892cb8cf483f3b6a17489be5057d,"History Teachers, Postsecondary",history teachers postsecondary,1.02,0.02
951,90c550c8805a100bd96937740668cbc8,Loan Counselors,loan counselors,1.01,0.04
952,7bbe67cb54b2e639a6a6e3fe505e54bb,Marriage and Family Therapists,marriage and family therapists,1.00,0.00


In [10]:
# Adding a threshold for importance
# We observe that this skills also fetches jobs like - marraige and family therapists and fire inspectors. Therefore we can set a threshold to get the right result.
# We can set the threhold by finding the mean value of the importance.

import numpy

mean_imp = numpy.mean(df_jobs.importance)
mean_imp

2.295419287211741

In [11]:
df_relavant_job = df_jobs[df_jobs.importance > mean_imp]
df_relavant_job

,job_uuid,job_title,normalized_job_title,importance,level
0,91e03483b47fc212441abba3acf5a4aa,Computer Hardware Engineers,computer hardware engineers,4.95,6.35
1,19a7f05a1ba98db9e72086ebc8bd3ae0,"Electronics Engineers, Except Computer",electronics engineers except computer,4.92,5.70
2,b8a5203799a5b6a9164b73170777fa11,Automotive Engineers,automotive engineers,4.91,6.39
3,351a52f8a687142e32acd0b6c331e61b,Aerospace Engineering and Operations Technicians,aerospace engineering and operations technicians,4.91,6.35
4,84f4c3f6e9e59cf9c4a840f6b5f2ebb3,Mechanical Engineers,mechanical engineers,4.89,6.08
...,...,...,...,...,...
400,a0027102cc17450c4af6a33f755169d4,Pathologists,pathologists,2.30,2.24
401,c75f2c543a800f4e10a793dbc80e77e0,Terrazzo Workers and Finishers,terrazzo workers and finishers,2.30,2.48
402,87fb6860c6331839f27031738b19e89d,Automotive Glass Installers and Repairers,automotive glass installers and repairers,2.30,2.17
403,f14be7795c5db8af8ddcad205219f2c8,Home Appliance Repairers,home appliance repairers,2.30,2.53


In [12]:
# Exporting the result to csv

df_relavant_job.to_csv('job_list.csv', mode='w', columns=['job_title'], index=False)

In [13]:
# SUMMARIZING IN A SINGLE Function

import requests
import json
import pandas as pd
from pandas import json_normalize
import numpy

def skill_job (skill):
  params = (('contains', skill),)
  response = requests.get('http://api.dataatwork.org/v1/skills/autocomplete', params=params)
  #print(response.url)
  response_json = response.json()
  df = pd.DataFrame(response_json)
  skillUUID = df.iloc[0,0]
  #print(skillUUID)
  response2 = requests.get("http://api.dataatwork.org/v1/skills/"+skillUUID+"/related_jobs")
  #print(response2.url)
  response2_json = response2.json()
  df2 = pd.DataFrame(response2_json)
  df_jobs = json_normalize(data=df2['jobs'])
  mean_imp = numpy.mean(df_jobs.importance)
  df_relavant_job = df_jobs[df_jobs.importance > mean_imp]
  print(str(len(df_relavant_job)) + " associated jobs found for the skill - " +skill)
  df_relavant_job.to_csv('job_list.csv', mode='w', columns=['job_title'], index=False)
  print("Final list is exported. Please find the file named as job_list.csv")


try:
  skill_job('engineering and technology')
except:
    print('Incorrect Skill Name. Please try again !')


405 associated jobs found for the skill - engineering and technology
Final list is exported. Please find the file named as job_list.csv
